## Reading dataset and Images

In [1]:
#  Functions for reading dataset and creating numpy array of images

import pandas as pd
import numpy as np
%matplotlib inline

from keras.preprocessing.image import load_img, img_to_array, array_to_img

Using TensorFlow backend.


In [2]:
# reading the dataset

data = pd.read_csv("./emergency_classification.csv")

FileNotFoundError: File b'./emergency_classification.csv' does not exist

In [ ]:
# checking head 

data.head()

In [ ]:
# checking number of emergency and non_emergency vehicles

data.emergency_or_not.value_counts()

In [ ]:
# reading all the images in numpy array

images = []
for img_loc in data.image_names:
    img = load_img('./images/' + img_loc)
    img = img_to_array(img)
    images.append(img)
    
images=np.array(images)

In [ ]:
images.shape

## Pre-processing images and creating validation dataset

In [ ]:
# pre-processing images

images = images/images.max()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# creating validation dataset

x_train, x_valid, y_train, y_valid = train_test_split(images, data['emergency_or_not'], test_size = 0.3, random_state = 42)

## Performing Image Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
    width_shift_range = 0.2,
    horizontal_flip = True
)

## Creating Model

In [ ]:
from keras.models import Sequential

In [ ]:
from keras.layers import InputLayer, Dense, Convolution2D, MaxPooling2D

In [ ]:
from keras.layers import BatchNormalization, GlobalAveragePooling2D, Dropout

In [ ]:
model = Sequential([
 InputLayer(input_shape=(224, 224, 3)),

 Convolution2D(32, (5, 5), activation='relu', padding='same'),
 MaxPooling2D(pool_size=2),
 BatchNormalization(),

 Convolution2D(64, (5, 5), activation='relu', padding='same'),
 Convolution2D(64, (5, 5), activation='relu', padding='same'),
 MaxPooling2D(pool_size=2),
 BatchNormalization(),

 Convolution2D(128, (5, 5), activation='relu', padding='same'),
 Convolution2D(128, (5, 5), activation='relu', padding='same'),
 MaxPooling2D(pool_size=2),
 BatchNormalization(),
    
 Convolution2D(256, (5, 5), activation='relu', padding='same'),
 Convolution2D(256, (5, 5), activation='relu', padding='same'),
 MaxPooling2D(pool_size=2),
 BatchNormalization(),
    
 Convolution2D(512, (5, 5), activation='relu', padding='same'),
 Convolution2D(512, (5, 5), activation='relu', padding='same'),

 GlobalAveragePooling2D(),
 BatchNormalization(),

 Dense(units=512, activation='sigmoid'),
 BatchNormalization(),
 Dropout(0.5),
    
 Dense(units=256, activation='sigmoid'),
 BatchNormalization(),
 Dropout(0.5),
    
 Dense(units=1, activation='sigmoid'),
])

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import SGD

In [ ]:
sgd = SGD(nesterov=True)

In [ ]:
import keras.backend as K
import tensorflow as tf

In [ ]:
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        eps = 1e-12
        y_pred=K.clip(y_pred,eps,1.-eps)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [ ]:
focal_loss = focal_loss()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
final_weights_path = 'best_cnn_model.h5'

callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor='val_loss', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_loss', patience=20, verbose=1)
]

In [ ]:
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),epochs=200,validation_data=(x_valid,y_valid), callbacks=callbacks_list, shuffle=False, workers=5, use_multiprocessing=True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plotting training accuracy vs validation accuracy

plt.plot(history.history['acc'], 'r')
plt.plot(history.history['val_acc'], 'b')
plt.legend({'training accuracy': 'r', 'validation accuracy':'b'})
plt.show()

In [ ]:
# Plotting training loss vs validation loss

plt.plot(history.history['loss'], 'r')
plt.plot(history.history['val_loss'], 'b')
plt.legend({'training loss': 'r', 'validation loss':'b'})
plt.show()